In [16]:
import gc
import matplotlib.pyplot as plt
plt.close('all')
gc.collect()
print("Memoria liberada.")

Memoria liberada.


In [ ]:
# =========================
# Indexación de Todos los Volúmenes para Preprocesamiento
# Indexa todos los pares imagen-label disponibles en TrainBatch1.
# MODIFICADA: Limita el procesamiento a 20 volúmenes para ahorrar espacio.
# =========================

import os
import numpy as np
from tqdm import tqdm
import nibabel as nib

# === 1. Configuración de rutas (relativas y solo TrainBatch1) ===
DATA_PATHS = {
    "images_batch1": r"./TrainBatch1/imagesTr",
    "labels_batch1": r"./TrainBatch1/labelsTr",
    # "images_batch2": r"./TrainBatch2/imagesTr",  # Comentado: no está disponible
    # "labels_batch2": r"./TrainBatch2/labelsTr"   # Comentado: no está disponible
}

# === 2. Validación de rutas ===
for key in ["images_batch1", "labels_batch1"]:
    path = DATA_PATHS[key]
    assert os.path.exists(path), f"Ruta no encontrada: {path}"

# === 3. Conteo y emparejamiento de archivos ===
images_path = DATA_PATHS["images_batch1"]
labels_path = DATA_PATHS["labels_batch1"]

image_files = sorted([f for f in os.listdir(images_path) if f.endswith(".nii.gz")])
label_files = sorted([f for f in os.listdir(labels_path) if f.endswith(".nii.gz")])

print(f">> Encontrados {len(image_files)} archivos de imagen y {len(label_files)} archivos de label.")

# Construir lista de pares (img_path, lbl_path)
all_pairs = []
for img_file in image_files:
    base_id = img_file.replace("_0000.nii.gz", "")
    # Buscar el label correspondiente
    matching_labels = [lbl for lbl in label_files if lbl.startswith(base_id)]
    if matching_labels:
        lbl_file = matching_labels[0]
        all_pairs.append((
            os.path.join(images_path, img_file),
            os.path.join(labels_path, lbl_file)
        ))
    else:
        print(f"Advertencia: No se encontró label para {img_file}")

print(f">> Emparejados {len(all_pairs)} pares imagen-label.")

# === 4. LIMITAR A 20 VOLÚMENES PARA AHORRAR ESPACIO ===
MAX_VOLUMES = 150
if len(all_pairs) > MAX_VOLUMES:
    print(f"Limitando el procesamiento a {MAX_VOLUMES} volúmenes (de {len(all_pairs)} disponibles) para ahorrar espacio.")
    all_pairs = all_pairs[:MAX_VOLUMES]
else:
    print(f"ℹProcesando todos los {len(all_pairs)} volúmenes disponibles.")

# === 5. Indexación: cargar metadatos de los volúmenes seleccionados ===
# Este paso es rápido porque solo carga el header, no los datos volumétricos.
loaded_data = {}
for img_path, lbl_path in tqdm(all_pairs, desc="Indexando volúmenes"):
    img_obj = nib.load(img_path, mmap=True)
    vol_id = os.path.basename(img_path).replace("_0000.nii.gz", "")
    
    loaded_data[vol_id] = {
        "img_path": img_path,
        "lbl_path": lbl_path,
        "affine": img_obj.affine,
        "header": img_obj.header.copy(),
        "zooms": tuple(img_obj.header.get_zooms()[:3])
    }
    img_obj.uncache()  # Libera la memoria

print(f"\nIndexación completada. {len(loaded_data)} volúmenes listos para su procesamiento.")

>> Encontrados 150 archivos de imagen y 150 archivos de label.
>> Emparejados 150 pares imagen-label.
ℹProcesando todos los 150 volúmenes disponibles.


Indexando volúmenes: 100%|██████████| 150/150 [00:01<00:00, 89.40it/s]


Indexación completada. 150 volúmenes listos para su procesamiento.


In [ ]:
# =========================
# Definición de Técnicas de Preprocesamiento
# Versión flexible que, con sus parámetros por defecto, replica el flujo de auditoría.
# CORREGIDA: sigma=0.8 para alinearse con la metodología validada.
# MODIFICADA: Normalización con rango fijo + reversión para pipeline masivo.
# =========================

import numpy as np

# -------------------------
# Helper: ensure ROI and volume have the same shape
# -------------------------
def match_shape(vol, roi):
    """Ajusta ROI para que coincida con la forma del volumen (crop/pad centrado)."""
    if roi is None:
        return None
    roi = roi.astype(bool, copy=False)
    Z, Y, X = vol.shape
    rz, ry, rx = roi.shape

    # Crop ROI si es más grande
    if rz > Z or ry > Y or rx > X:
        z0 = max((rz - Z) // 2, 0)
        y0 = max((ry - Y) // 2, 0)
        x0 = max((rx - X) // 2, 0)
        roi = roi[z0:z0+Z, y0:y0+Y, x0:x0+X]

    # Pad ROI si es más pequeña
    diffZ = Z - roi.shape[0]
    diffY = Y - roi.shape[1]
    diffX = X - roi.shape[2]
    if diffZ > 0 or diffY > 0 or diffX > 0:
        pad_width = (
            (0, max(diffZ, 0)),
            (0, max(diffY, 0)),
            (0, max(diffX, 0)),
        )
        roi = np.pad(roi, pad_width, mode="constant", constant_values=False)

    return roi[:Z, :Y, :X]

# -------------------------
# 1) HU Clipping
# -------------------------
def apply_hu_clipping(volume, hu_min=-1024, hu_max=600):
    """
    Aplica clipping en HU al rango [hu_min, hu_max].
    """
    return np.clip(volume.astype(np.float32, copy=False), hu_min, hu_max)

# -------------------------
# 2) Gaussian Smoothing
# -------------------------
def apply_gaussian_smoothing(volume, sigma=0.8):
    """
    Aplica suavizado gaussiano isotrópico.
    - sigma=0.8: un valor ligeramente más conservador que 1.0 para preservar vías finas.
    """
    from scipy.ndimage import gaussian_filter
    return gaussian_filter(volume.astype(np.float32), sigma=sigma)

# -------------------------
# 3) Normalización Min-Max con Rango Fijo + Reversión
# -------------------------
# Diccionario global para guardar los parámetros de normalización por caso.
_NORMALIZATION_PARAMS = {}

def apply_minmax_normalization(volume, case_id="default"):
    """
    Aplica normalización Min-Max al rango [0, 1] usando el rango fijo [-1024, 600].
    Guarda los parámetros para su reversión.
    """
    vol_float = volume.astype(np.float32, copy=False)
    hu_min, hu_max = -1024.0, 600.0  # Rango fijo
    normalized = (vol_float - hu_min) / (hu_max - hu_min)
    # Guardar los parámetros (siempre los mismos, pero por consistencia)
    _NORMALIZATION_PARAMS[case_id] = (hu_min, hu_max)
    return normalized

def revert_normalization(normalized_volume, case_id="default"):
    """
    Revierte la normalización Min-Max usando el rango fijo [-1024, 600].
    """
    hu_min, hu_max = -1024.0, 600.0  # Rango fijo
    return normalized_volume * (hu_max - hu_min) + hu_min

# -------------------------
# 4) Padding (symmetrical, multiples of 32)
# -------------------------
def apply_padding_32(volume, pad_value=-1024):
    """
    Aplica padding simétrico para ajustar a múltiplos de 32.
    Usa el valor de aire exterior (-1024) como constante de relleno.
    """
    shape = volume.shape
    pad_width = []
    for dim in shape:
        remainder = dim % 32
        if remainder == 0:
            pad_before, pad_after = 0, 0
        else:
            pad_total = 32 - remainder
            pad_before = pad_total // 2
            pad_after = pad_total - pad_before
        pad_width.append((pad_before, pad_after))
    
    padded = np.pad(volume, pad_width, mode='constant', constant_values=pad_value)
    return padded.astype(np.float32, copy=False)

In [ ]:
# =========================
# Generación de UNA Combinación Específica para Todos los Volúmenes
# Aplica UNA combinación de preprocesamiento a todos los volúmenes indexados.
# CORREGIDA: Orden lógico + Verificación de existencia + case_id en normalización.
# =========================

import os
import itertools
from tqdm import tqdm
import nibabel as nib
import numpy as np

# === 1. SELECCIONAR LA COMBINACIÓN ESPECÍFICA ===
# Opciones disponibles (elige UNA):
COMBINACION_SELECCIONADA = "Clipped_Smoothed_Normalized_Padded"  

# Mapeo de nombres a configuraciones
combinacion_config = {
    "Clipped": [0],
    "Smoothed": [1], 
    "Normalized": [2],
    "Padded": [3],
    "Clipped_Smoothed": [0, 1],
    "Clipped_Normalized": [0, 2],
    "Clipped_Padded": [0, 3],
    "Smoothed_Normalized": [1, 2],
    "Smoothed_Padded": [1, 3],
    "Normalized_Padded": [2, 3],
    "Clipped_Smoothed_Normalized": [0, 1, 2],
    "Clipped_Smoothed_Padded": [0, 1, 3],
    "Clipped_Normalized_Padded": [0, 2, 3],
    "Smoothed_Normalized_Padded": [1, 2, 3],
    "Clipped_Smoothed_Normalized_Padded": [0, 1, 2, 3]
}

if COMBINACION_SELECCIONADA not in combinacion_config:
    raise ValueError(f"❌ Combinación '{COMBINACION_SELECCIONADA}' no válida.\n"
                    f"Opciones disponibles: {list(combinacion_config.keys())}")

combo_subset = combinacion_config[COMBINACION_SELECCIONADA]
print(f"Combinación seleccionada: {COMBINACION_SELECCIONADA}")
print(f"   Técnicas: {[['Clipped','Smoothed','Normalized','Padded'][i] for i in combo_subset]}")

# === 2. Configuración de la ruta base de salida ===
OUTPUT_BASE_DIR = r"./PreProce_All"
os.makedirs(OUTPUT_BASE_DIR, exist_ok=True)
print(f"\nLas combinaciones se guardarán en: {os.path.abspath(OUTPUT_BASE_DIR)}")

# === 3. Crear carpeta para la combinación seleccionada ===
folder_name = COMBINACION_SELECCIONADA
combo_output_dir = os.path.join(OUTPUT_BASE_DIR, folder_name)
os.makedirs(combo_output_dir, exist_ok=True)
print(f"Carpeta creada: {folder_name}")

# === 4. Procesar cada volumen para la combinación seleccionada ===
print(f"\nIniciando el procesamiento de {len(loaded_data)} volúmenes...")

total_saved = 0
failed_volumes = []
skipped_volumes = 0

for vol_id, meta in tqdm(loaded_data.items(), desc=f"Procesando {COMBINACION_SELECCIONADA}"):
    try:
        output_filename = os.path.basename(meta["img_path"])
        vol_output_dir = os.path.join(combo_output_dir, vol_id)
        output_path = os.path.join(vol_output_dir, output_filename)
        
        # Verificar si ya existe
        if os.path.exists(output_path):
            skipped_volumes += 1
            continue
        
        # Cargar el volumen original
        img_obj = nib.load(meta["img_path"])
        vol_orig = img_obj.get_fdata(dtype=np.float32)
        affine, header = img_obj.affine, img_obj.header
        
        # Aplicar la combinación seleccionada
        vol = vol_orig.copy()
        
        # Aplicar técnicas en orden lógico CORREGIDO:
        # Clipping → Suavizado → Padding → Normalización
        if 0 in combo_subset:  # Clipped
            vol = apply_hu_clipping(vol)
        if 1 in combo_subset:  # Smoothed
            vol = apply_gaussian_smoothing(vol)
        if 3 in combo_subset:  # Padded
            vol = apply_padding_32(vol)  # Padding en escala HU original
        if 2 in combo_subset:  # Normalized
            vol = apply_minmax_normalization(vol, case_id=vol_id)  # ← ¡¡¡AÑADIDO case_id=vol_id!!!
        
        # Guardar el volumen preprocesado
        os.makedirs(vol_output_dir, exist_ok=True)
        output_nii = nib.Nifti1Image(vol, affine=affine, header=header)
        nib.save(output_nii, output_path)
        total_saved += 1
        
        img_obj.uncache()
        del vol_orig, vol, output_nii
        
    except Exception as e:
        print(f"\nError al procesar {vol_id}: {str(e)}")
        failed_volumes.append(vol_id)

# === 5. Resumen final ===
processed_volumes = len(loaded_data) - len(failed_volumes) - skipped_volumes

print(f"\n{'='*70}")
print(f"RESUMEN DE LA GENERACIÓN: {COMBINACION_SELECCIONADA}")
print(f"{'='*70}")
print(f"Total de volúmenes procesados: {processed_volumes}")
print(f"Total de volúmenes saltados (ya existentes): {skipped_volumes}")
print(f"Total de archivos generados: {total_saved}")
print(f"Volúmenes fallidos: {len(failed_volumes)}")

if failed_volumes:
    print("\nVolúmenes fallidos:")
    for vol in failed_volumes:
        print(f" - {vol}")

if skipped_volumes > 0:
    print(f"\nℹ{skipped_volumes} volúmenes ya existían y fueron saltados.")

print(f"\nProcesamiento completado para: {COMBINACION_SELECCIONADA}")

Combinación seleccionada: Clipped_Smoothed_Normalized_Padded
   Técnicas: ['Clipped', 'Smoothed', 'Normalized', 'Padded']

Las combinaciones se guardarán en: c:\Users\pipea\OneDrive\Escritorio\PIB\PreProce_All
Carpeta creada: Clipped_Smoothed_Normalized_Padded

Iniciando el procesamiento de 150 volúmenes...


Procesando Clipped_Smoothed_Normalized_Padded: 100%|██████████| 150/150 [1:05:30<00:00, 26.20s/it]


RESUMEN DE LA GENERACIÓN: Clipped_Smoothed_Normalized_Padded
Total de volúmenes procesados: 150
Total de volúmenes saltados (ya existentes): 0
Total de archivos generados: 150
Volúmenes fallidos: 0

Procesamiento completado para: Clipped_Smoothed_Normalized_Padded


In [ ]:
# =========================
# Generación de Vías Aéreas para UNA Combinación Específica
# USA LAS ROIS PREEXISTENTES DE ./rois_auto/ Y EL PIPELINE TUBULAR COMPLETO.
# MODIFICADA: Procesa solo la combinación seleccionada + validación + reversión.
# =========================

import os
import nibabel as nib
import numpy as np
from skimage.morphology import ball
from scipy.ndimage import (
    label as cc_label, 
    binary_dilation, 
    binary_closing
)
from collections import deque
from tqdm import tqdm

# === SELECCIONAR LA MISMA COMBINACIÓN QUE EN CELL 3 ===
COMBINACION_SELECCIONADA = "Clipped_Smoothed_Normalized_Padded"

print(f"Procesando vías aéreas para: {COMBINACION_SELECCIONADA}")

# === Función: Pipeline de Vías Aéreas (IDÉNTICO a Cell 4 TUBULAR) ===
def generate_airways_tubular(vol_clip, final_roi):
    """Genera máscara de vías aéreas usando el pipeline TUBULAR exacto de Cell 4."""    
    roi_hu_values = vol_clip[final_roi > 0]
    if roi_hu_values.size == 0:
        return np.zeros_like(vol_clip, dtype=np.uint8)
    
    # --- Step 0: Umbrales adaptativos más estrictos ---
    GROW_THR = np.percentile(roi_hu_values, 3)
    AIR_THR_STRICT = min(-980, np.percentile(roi_hu_values, 0.5))

    # --- Step 1-3: Segmentación inicial con BFS (sin tolerancia) ---
    seeds = (vol_clip <= AIR_THR_STRICT) & (final_roi > 0)
    Z, Y, X = vol_clip.shape
    top = np.zeros_like(seeds, dtype=bool)
    top[:int(Z * 0.5)] = True
    trachea_cand = seeds & top
    labeled_top, n_top = cc_label(trachea_cand)
    if n_top > 0:
        sizes_top = np.bincount(labeled_top.ravel())
        sizes_top[0] = 0
        trachea_label = np.argmax(sizes_top)
        trachea = labeled_top == trachea_label
    else:
        trachea = np.zeros_like(seeds, dtype=bool)

    airways_mask_bfs = np.zeros_like(seeds, dtype=np.uint8)
    q = deque()
    seed_pts = np.argwhere(trachea)
    for s in seed_pts:
        airways_mask_bfs[tuple(s)] = 1
        q.append(tuple(s))

    def neighbors6(z, y, x, shape):
        for dz, dy, dx in [(1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1)]:
            nz, ny, nx = z + dz, y + dy, x + dx
            if 0 <= nz < shape[0] and 0 <= ny < shape[1] and 0 <= nx < shape[2]:
                yield nz, ny, nx

    # BFS estricto
    while q:
        z, y, x = q.popleft()
        for nz, ny, nx in neighbors6(z, y, x, airways_mask_bfs.shape):
            if airways_mask_bfs[nz, ny, nx] == 0 and final_roi[nz, ny, nx] > 0:
                neighbor_val = vol_clip[nz, ny, nx]
                if neighbor_val <= GROW_THR:
                    airways_mask_bfs[nz, ny, nx] = 1
                    q.append((nz, ny, nx))

    # Post-procesamiento inicial
    airways_mask = airways_mask_bfs.astype(bool)
    airways_mask = binary_dilation(airways_mask, iterations=1)
    airways_mask = binary_closing(airways_mask, structure=ball(2))

    # --- Step 4: Filtrado por forma tubular ---
    labeled_mask, n_components = cc_label(airways_mask)
    if n_components > 1:
        component_sizes = np.bincount(labeled_mask.ravel())
        component_sizes[0] = 0
        main_component_id = np.argmax(component_sizes)
        valid_mask = np.zeros_like(airways_mask, dtype=bool)
        valid_mask[labeled_mask == main_component_id] = True  # Mantener componente principal
        
        for i in range(1, n_components + 1):
            if i == main_component_id:
                continue
            comp_mask = labeled_mask == i
            size = np.sum(comp_mask)
            # Filtrar por tamaño
            if size < 30 or size > 50000:
                continue
            
            # Calcular bounding box para análisis de forma
            coords = np.argwhere(comp_mask)
            if len(coords) == 0:
                continue
            z_min, y_min, x_min = coords.min(axis=0)
            z_max, y_max, x_max = coords.max(axis=0)
            dz = max(1, z_max - z_min)
            dy = max(1, y_max - y_min)
            dx = max(1, x_max - x_min)
            dimensions = np.sort([dz, dy, dx])
            
            # Criterio tubular: la dimensión más larga debe ser al menos 2x la más corta
            if dimensions[2] >= 2 * dimensions[0]:
                valid_mask[comp_mask] = True
        
        airways_mask = valid_mask

    # --- Step 5: Extensión iterativa para vías finas ---
    extended_mask = airways_mask.copy()
    current_frontier = airways_mask.copy()
    thresholds = [-890, -400, 0]
    for current_thr in thresholds:
        next_frontier = np.zeros_like(extended_mask)
        dilated = binary_dilation(current_frontier, iterations=1)
        frontier = dilated & (~extended_mask)
        for z, y, x in np.argwhere(frontier):
            if final_roi[z, y, x] > 0 and vol_clip[z, y, x] <= current_thr:
                extended_mask[z, y, x] = 1
                next_frontier[z, y, x] = 1
        current_frontier = next_frontier
        if not np.any(current_frontier):
            break

    # --- Step 6: Resultado final ---
    return extended_mask.astype(np.uint8)

# === Configuración de rutas ===
INPUT_BASE_DIR = r"./PreProce_All"
ROIS_AUTO_DIR = r"./rois_auto"  # Carpeta con ROIs pregeneradas

# Verificar que la carpeta de ROIs existe
if not os.path.exists(ROIS_AUTO_DIR):
    raise FileNotFoundError(f"Carpeta de ROIs no encontrada: {os.path.abspath(ROIS_AUTO_DIR)}")

# Verificar que la combinación seleccionada existe
combo_path = os.path.join(INPUT_BASE_DIR, COMBINACION_SELECCIONADA)
if not os.path.exists(combo_path):
    raise FileNotFoundError(f"Combinación '{COMBINACION_SELECCIONADA}' no encontrada en {INPUT_BASE_DIR}")

print(f"\nRuta de combinación: {combo_path}")

# === Procesar solo la combinación seleccionada ===
total_processed = 0
failed_cases = []
skipped_existing = 0

print(f"\nProcesando combinación: {COMBINACION_SELECCIONADA}")

# Recorrer cada carpeta de volumen dentro de la combinación seleccionada
vol_folders = [d for d in os.listdir(combo_path) if os.path.isdir(os.path.join(combo_path, d))]
for vol_id in tqdm(vol_folders, desc=f"  Volúmenes en {COMBINACION_SELECCIONADA}"):
    vol_path = os.path.join(combo_path, vol_id)
    
    # Encontrar el archivo de volumen
    nii_files = [f for f in os.listdir(vol_path) if f.endswith(".nii.gz")]
    if not nii_files:
        continue
    vol_filename = nii_files[0]
    vol_file_path = os.path.join(vol_path, vol_filename)
    
    # === VERIFICAR SI LA MÁSCARA DE VÍAS AÉREAS YA EXISTE ===
    airways_filename = vol_filename.replace("_0000.nii.gz", "_airways.nii.gz")
    airways_path = os.path.join(vol_path, airways_filename)
    
    if os.path.exists(airways_path):
        total_processed += 1
        skipped_existing += 1
        continue  # Saltar si ya existe
    
    try:
        # Cargar el volumen preprocesado
        img = nib.load(vol_file_path)
        vol_preproc = img.get_fdata(dtype=np.float32)
        affine = img.affine
        
        # === Cargar la ROI preexistente ===
        roi_path = os.path.join(ROIS_AUTO_DIR, f"{vol_id}.nii.gz")
        if not os.path.exists(roi_path):
            error_msg = f"ROI no encontrada para {vol_id}"
            failed_cases.append(f"{COMBINACION_SELECCIONADA}/{vol_id}: {error_msg}")
            continue
            
        roi_img = nib.load(roi_path)
        final_roi_original = roi_img.get_fdata().astype(np.uint8)
        
        # === Manejo del padding: ajustar la ROI si es necesario ===
        if "Padded" in COMBINACION_SELECCIONADA:
            from scipy.ndimage import zoom
            vol_shape = vol_preproc.shape
            roi_shape = final_roi_original.shape
            
            # Calcular factores de zoom
            zoom_factors = [vol_shape[i] / roi_shape[i] for i in range(3)]
            final_roi_for_vol = zoom(final_roi_original.astype(float), zoom_factors, order=0)
            final_roi_for_vol = (final_roi_for_vol > 0.5).astype(np.uint8)
            
            if final_roi_for_vol.shape != vol_shape:
                error_msg = f"Formas no coinciden después del padding"
                failed_cases.append(f"{COMBINACION_SELECCIONADA}/{vol_id}: {error_msg}")
                continue
                
            roi_for_segmentation = final_roi_for_vol
            vol_for_segmentation = vol_preproc
        else:
            # Verificar que las formas coincidan
            if vol_preproc.shape != final_roi_original.shape:
                error_msg = f"Formas no coinciden: vol={vol_preproc.shape}, roi={final_roi_original.shape}"
                failed_cases.append(f"{COMBINACION_SELECCIONADA}/{vol_id}: {error_msg}")
                continue
                
            roi_for_segmentation = final_roi_original
            vol_for_segmentation = vol_preproc
        
        # === ¡¡¡AÑADIR REVERSIÓN DE NORMALIZACIÓN!!! ===
        if "Normalized" in COMBINACION_SELECCIONADA:
            vol_for_segmentation = revert_normalization(vol_for_segmentation, case_id=vol_id)
        
        # === Generar vías aéreas usando el pipeline TUBULAR exacto ===
        airways_mask = generate_airways_tubular(vol_for_segmentation, roi_for_segmentation)
        
        # Guardar la máscara en la misma carpeta del volumen
        airways_nii = nib.Nifti1Image(airways_mask, affine=affine)
        nib.save(airways_nii, airways_path)
        
        total_processed += 1
        
    except Exception as e:
        error_msg = f"Error de procesamiento: {str(e)}"
        failed_cases.append(f"{COMBINACION_SELECCIONADA}/{vol_id}: {error_msg}")

# === Resumen final ===
print(f"\n{'='*70}")
print(f"RESUMEN DE LA GENERACIÓN DE VÍAS AÉREAS: {COMBINACION_SELECCIONADA}")
print(f"{'='*70}")
print(f"Total de máscaras generadas: {total_processed - skipped_existing}")
print(f"Total de máscaras ya existentes: {skipped_existing}")
print(f"Total procesado (incluyendo existentes): {total_processed}")
print(f"Total de casos fallidos: {len(failed_cases)}")

if skipped_existing > 0:
    print(f"\nℹ{skipped_existing} volúmenes ya tenían máscaras y fueron saltados.")

if failed_cases:
    print("\nCasos fallidos:")
    for err in failed_cases:
        print(f" - {err}")

print(f"\nGeneración de vías aéreas completada para: {COMBINACION_SELECCIONADA}")

Procesando vías aéreas para: Clipped_Smoothed_Normalized_Padded

Ruta de combinación: ./PreProce_All\Clipped_Smoothed_Normalized_Padded

Procesando combinación: Clipped_Smoothed_Normalized_Padded


  Volúmenes en Clipped_Smoothed_Normalized_Padded: 100%|██████████| 150/150 [2:10:30<00:00, 52.20s/it] 


RESUMEN DE LA GENERACIÓN DE VÍAS AÉREAS: Clipped_Smoothed_Normalized_Padded
Total de máscaras generadas: 150
Total de máscaras ya existentes: 0
Total procesado (incluyendo existentes): 150
Total de casos fallidos: 0

Generación de vías aéreas completada para: Clipped_Smoothed_Normalized_Padded
